# Polarsに関するTips

Polarsは、データフレーム操作を効率的に行うための強力なライブラリですが、さらに効果的に活用するためにはいくつかの便利なテクニックやコツを知っておくことが重要です。この章では、Polarsをよりスムーズに、かつ効率的に使用するための実践的なTipsを紹介します。これらのテクニックを使うことで、データの操作や処理を一層高速化し、作業の生産性を向上させることができます。

In [1]:
import panel as pn
pn.extension()
import polars as pl
from helper.jupyter import row

## メッセージチェン

Polarsでは、メソッドチェーンを活用することでデータ操作を効率的に行うことができますが、メソッドチェーンの途中でどのようにデータが変化するのかを把握するのは難しい場合があります。そこで、デバッグのために次の2つの方法を活用できます。

* `helper.polars.DataCapturer`: メソッドチェーンの任意の点でデータをキャプチャし、状態を確認できます。
* `helper.polars.PipeLogger`: メソッドチェーン全体の入出力と引数をキャプチャし、可視化します。

### DataCapturer

`DataCapturer`を使用すると、メソッドチェーン内の特定のポイントでデータの状態をキャプチャできます。以下は、`DataCapturer`の使い方を示す例です。

In [2]:
from helper.polars import DataCapturer

df = pl.DataFrame({
    "name": ["Alice", "Bob", "Charlie", "David", "Eve"],
    "age": [25, 30, 35, 40, 45],
    "score": [85, 90, 95, 80, 70],
    "department": ["HR", "IT", "HR", "IT", "HR"]
})

cap = DataCapturer() #❶

result = (
df
.filter(pl.col("age") > 30)  
.with_columns(
  (pl.col("score") * 1.1).alias("adjusted_score")
)
.pipe(cap.before_group) #❷
.group_by("department")
.agg([
  pl.col("adjusted_score").mean().alias("average_score"),
  pl.col("age").max().alias("max_age")
])
.pipe(cap.before_sort) #❷
.sort("average_score", descending=True)
)

row(cap.before_group, cap.before_sort) #❸

shape: (3, 5) name age score department adjusted_score str i64 i64 str f64 "Charlie" 35 95 "HR" 104.5 "David" 40 80 "IT" 88.0 "Eve" 45 70 "HR" 77.0 
 shape: (2, 3) department average_score max_age str f64 i64 "HR" 90.75 45 "IT" 88.0 40

このコードでは、`cap.before_group`でグループ化の前のデータ状態をキャプチャし、`cap.before_sort`でソート前のデータ状態をキャプチャしています。

❶メソッドチェーンを開始する前に、`DataCapturer()`のインスタンス`cap`を作成します。<br>
❷メソッドチェーンの特定の場所で、`.pipe(cap.name)`を挿入し、その時点のデータを`name`としてキャプチャします。<br>
❸キャプチャしたデータは`cap.name`でアクセスできます。<br>

### PipeLogger

`PipeLogger`を使用すると、メソッドチェーン内の各ステップで発生するデータの入出力を視覚的に確認できます。これにより、データがどのように変化しているかを逐一確認しながら処理を進めることができ、デバッグが容易になります。

以下のコード例では、`PipeLogger`を使用してデータフレームを操作し、処理中の入出力をJupyterLabで可視化する方法を示します。

In [3]:
from helper.polars import PipeLogger

result = (
PipeLogger(df) #❶
.filter(pl.col("age") > 30)  # 年齢が30歳以上の行をフィルタリング
.with_columns(
  (pl.col("score") * 1.1).alias("adjusted_score")  # スコアを調整
)
.group_by("department")  # 部署ごとにグループ化
.agg([
  pl.col("adjusted_score").mean().alias("average_score"),  # 各部署の平均スコアを計算
  pl.col("age").max().alias("max_age")  # 各部署の最年長者の年齢を取得
])
.sort("average_score", descending=True)  # 平均スコアで降順ソート
)

result #❷

`PipeLogger`を使うことで、メソッドチェーン内のデータの変化を視覚的に追跡し、デバッグがしやすくなります。

❶最初のオブジェクトである`df`を`PipeLogger()`でラップします。これにより、結果は`DataFrame`ではなく、すべての入出力をキャプチャする`PipeLogger`オブジェクトになります。`PipeLogger`は、メソッドチェーンの各ステップでデータの状態を保存します。

❷最後に、`PipeLogger`オブジェクトをセルの最後に配置し、インタラクティブなウィジェットとして結果を表示します。これにより、各メソッドの入出力を可視化することができます。

## 複雑な演算式

同じ中間結果を複数回利用する場合、メソッドチェーンのメリットが薄れることがあります。このような場合は、中間結果を変数に保存して後で利用するのが一般的です。たとえば、3次元ベクトルを正規化（normalize）するとき、ベクトルの長さを複数回使用します。一見すると、`length`の式が複数回実行されるように見えますが、Polarsでは演算結果をキャッシュするため、同じ計算を2回実行することはありません。

In [2]:
df = pl.DataFrame(
    dict(x=[1.0, 2.0, 3.0], y=[3.0, 2.0, 1.0], z=[2.0, 1.0, 3.0])
)

x, y, z = pl.col('x'), pl.col('y'), pl.col('z')
length = (x**2 + y**2 + z**2).sqrt()

df1 = df.select(
    x = x / length,
    y = y / length,
    z = z / length
)

row(df, df1)

x,y,z
f64,f64,f64
1.0,3.0,2.0
2.0,2.0,1.0
3.0,1.0,3.0
x,y,z
f64,f64,f64
0.267261,0.801784,0.534522
0.666667,0.666667,0.333333
0.688247,0.229416,0.688247


次のコードでは、`LazyDataFrame.explain()`を使用して実行プランを表示します。この結果、ベクトルの長さを計算する式が`__POLARS_CSER_0x30925ccb05afdfe7`という名前の列にキャッシュされていることが分かります。

In [5]:
print(
    df
    .lazy()
    .with_columns(
        x = x / length,
        y = y / length,
        z = z / length
    )
    .explain()
)

simple π 3/4 ["x", "y", "z"]
   WITH_COLUMNS:
   [[(col("x")) / (col("__POLARS_CSER_0x30925ccb05afdfe7"))].alias("x"), [(col("y")) / (col("__POLARS_CSER_0x30925ccb05afdfe7"))].alias("y"), [(col("z")) / (col("__POLARS_CSER_0x30925ccb05afdfe7"))].alias("z")] 
     WITH_COLUMNS:
     [[([(col("x").pow([dyn int: 2])) + (col("y").pow([dyn int: 2]))]) + (col("z").pow([dyn int: 2]))].sqrt().alias("__POLARS_CSER_0x30925ccb05afdfe7")] 
      DF ["x", "y", "z"]; PROJECT */3 COLUMNS; SELECTION: None


直接演算式で複雑な計算を書くのは非常に手間がかかります。例えば、平方根を求める場合、`sqrt(x)`のように記述することはできず、`x.sqrt()`と書く必要があります。より数学的な記法で演算式を記述できるようにするため、本書では次の`polars_exprs`デコレータを提供します。このデコレータを使用することで、通常の関数をPolarsの演算式に変換することができます。この関数内では、演算式の利点をそのまま活用することが可能です。また、`.list.first()`のようなサブネームスペースの操作も、`list_first()`のような関数として利用できます。さらに、`c_`で始まる変数は列を表す演算式として解釈されます。例えば、`c_x`は`pl.col('x')`と同じ意味を持ちます。

In [6]:
from helper.polars import polars_exprs

@polars_exprs
def norm():
    length = sqrt(c_x**2 + c_y**2 + c_z**2)
    return dict(x=x / length, y=y / length, z=z / length)

df2 = df.select(**norm())
row(df, df2)

x,y,z
f64,f64,f64
1.0,3.0,2.0
2.0,2.0,1.0
3.0,1.0,3.0
x,y,z
f64,f64,f64
0.267261,0.801784,0.534522
0.666667,0.666667,0.333333
0.688247,0.229416,0.688247


`c_`で始まる変数名を使用しない場合、キーワード引数を用いて列名を指定することができます。例えば、次のコードではその例を示しています：

In [17]:
@polars_exprs
def norm2():
    length = sqrt(x**2 + y**2 + z**2)
    return dict(x=x / length, y=y / length, z=z / length)

df = pl.DataFrame(
    dict(a=[1.0, 2.0, 3.0], b=[3.0, 2.0, 1.0], c=[2.0, 1.0, 3.0])
)
df1 = df.select(**norm2(x='a', y='b', z='c'))

row(df, df1)

a,b,c
f64,f64,f64
1.0,3.0,2.0
2.0,2.0,1.0
3.0,1.0,3.0
x,y,z
f64,f64,f64
0.267261,0.801784,0.534522
0.666667,0.666667,0.333333
0.688247,0.229416,0.688247


`polars_exprs`でデコレートされた関数が実行される際、以下の順序でシンボルが解釈されます。

1. キーワード引数: キーワード引数に指定されたシンボルは、そのままキーワード引数として解釈されます。引数の値が文字列の場合、自動的に`pl.col()`を用いて列を表す式に置き換えられます。  
2. `c_`で始まるシンボル: `c_`で始まるシンボルは`pl.col()`に変換されます。例えば、`c_name`は`pl.col('name')`となります。  
3. Polarsライブラリ内の演算式関連の関数やメソッド
4. 関数定義時のグローバル変数
5. Pythonのビルトイン関数

以下の例では、それぞれのシンボルが次のように解釈されます：

- **`A`と`p`**：キーワード引数で指定した列。`pl.col('Amp')`および`pl.col('p')`に対応します。
- **`c_f`**：列`pl.col('f')`に変換されます。
- **`math`**：グローバル変数として解釈されます。
- **`sin()`**：Polarsの演算式メソッド`sin()`を使用します。
- **`print()`**：Pythonのビルトイン関数`print()`を使用します。

In [25]:
import math

@polars_exprs
def my_expr():
    phase = 2 * math.pi * c_f + p
    print(phase)
    value = A * sin(phase)
    return value

df = pl.DataFrame(dict(
    Amp=[1, 2, 3],
    f=[10, 20, 30],
    p=[0.1, 0.2, 0.3]
))

df.select(my_expr(A="Amp", p="p"))

[([(dyn float: 6.283185) * (col("f"))]) + (col("p"))]


Amp
f64
0.099833
0.397339
0.886561


`polars_exprs`デコレータを使用することで得られる主なメリットは以下の通りです：

1. **より直感的な記法で計算式を記述できる**
   - 通常のPython関数のように計算式を書けるため、コードが簡潔で分かりやすくなります。
   - 例えば、`sqrt(x)`のような数学的な記法で演算式を記述可能です（通常のPolarsでは`x.sqrt()`と記述する必要があります）。

2. **列参照が簡単になる**
   - `c_`で始まる変数（例: `c_x`）を使えば、`pl.col('x')`のように書かずに列を参照できます。
   - キーワード引数を使うことで、柔軟に列名を指定可能です。例えば、`x='column1'`のように明示的に列を指定できます。

3. **Polars演算式のメリットをそのまま活用**
   - 関数内で使用される演算はPolarsの遅延評価（lazy execution）や高速なクエリ実行エンジンの恩恵を受けられるため、大量データの処理でも効率的です。
   - 演算式の関数やメソッド（例: `sin()`や`sqrt()`）が直接使えるため、記述が直感的になります。

4. **サブネームスペースの関数が簡単に使える**
   - Polarsのサブネームスペース（例: `.list.first()`）に対応したカスタム関数（例: `list_first()`）を使えるため、柔軟性が向上します。

5. **コードの再利用性が向上**
   - 演算式をPython関数として抽象化できるため、複雑な計算や処理を繰り返し使用する場合にコードを再利用しやすくなります。
   - 同じ計算式を異なる列に適用したい場合も、キーワード引数を用いて簡単に対応可能です。

6. **Polarsの機能とPythonの標準機能の統合**
   - Polars演算式に加え、標準Pythonのビルトイン関数（例: `print()`）やグローバル変数（例: `math.pi`）も利用できるため、柔軟な処理が可能です。

7. **読みやすく保守性の高いコード**
   - 数学的な記法や柔軟な列指定によって、コードが読みやすくなるため、保守性が向上します。
   - チームでの共有や将来的な修正が容易になります。

## 配列を処理するユーザー関数

次のコードは、`a` 列が `b` 列より小さい値を指定された値に置き換えます。Polars の演算式でこの計算を実装するのは難しいため、`pyarrow` や `numpy` の機能を使用すれば簡単に実現でき、`map_batches()` を活用しています。

In [5]:
import numpy as np

df = pl.DataFrame({
    "a":[1, 2, 3, 10],
    "b":[1, 3, 4, 5],
})

def func(args, values):
    a, b = [s.to_numpy() for s in args]
    c = np.copy(a)
    c[a < b] = values
    return pl.Series(c)

df.with_columns(
    pl_result=pl.map_batches(['a', 'b'], lambda args:func(args, [100, 200]))
)    

a,b,pl_result
i64,i64,i64
1,1,1
2,3,100
3,4,200
10,5,10


`map_batches()` には次の三つの問題があります。

1. 複数の列を一つのリストにまとめて、ユーザー関数に渡します。
2. ユーザー関数には他の引数を渡せません。
3. ユーザー関数の入力と出力が `Series` オブジェクトでないと、結果がおかしくなります。

これらの問題を解決するために、本書では `pyarrow_batch` と `numpy_batch` の二つのデコレータを提供します。次のコードは、`pyarrow_batch` と `numpy_batch` を使って、`pa_func` と `np_func` を `map_batches()` に渡せる関数に変換します。また、これらの関数に引数を渡したい場合は、`func(values=values)` のように使うことができます。

In [7]:
from helper.polars import pyarrow_batch, numpy_batch

@pyarrow_batch
def pa_func(a, b, values):
    from pyarrow.compute import replace_with_mask, less
    return replace_with_mask(a, less(a, b), values)

@numpy_batch
def np_func(a, b, values):
    c = a.copy()
    c[a < b] = values
    return c

values = [100, 200]

df.with_columns(
    pa_result=pl.map_batches(['a', 'b'], pa_func(values=values)),
    np_result=pl.map_batches(['a', 'b'], np_func(values=values)),
)

a,b,pa_result,np_result
i64,i64,i64,i64
1,1,1,1
2,3,100,100
3,4,200,200
10,5,10,10
